In [1]:
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd

In [2]:
df = dd.read_csv('9_23_hotness_01_percent.csv', assume_missing = True, blocksize = 64e6)

In [3]:
df.head()

,GUID,SESSION_SKEY,SITE_ID,SESSION_START_DT,ITEM_ID,MIN_SEQNUM,MIN_EVENT_TIMESTAMP,USER_ID,PLATFORM,asp_usd,...,CATEG_GMB_P1D_BEFORE_VI,TTL_BI_P1D_BEFORE_VI,TTL_GMB_P1D_BEFORE_VI,ITEM_BI_SHARE_IN_CATEG_P1D_BEFORE_VI,ITEM_GMB_SHARE_IN_CATEG_P1D_BEFORE_VI,CATEG_BI_SHARE_IN_TTL_P1D_BEFORE_VI,CATEG_GMB_SHARE_IN_TTL_P1D_BEFORE_VI,BI_INVERSE_FREQUENCY_SCORE,GMB_INVERSE_FREQUENCY_SCORE,rnk
0,9937d3881710a6e6f02cd2e4ff7ad788,3.805118e+13,3.0,2020-07-30,1.311577e+11,43.0,2020-07-30 17:15:51,0.000000e+00,Browser: mWeb,5.0,...,1.538424e+05,6525144.0,2.188364e+08,0.00017,0.000218,0.001804,0.000703,0.001074,0.001582,0.006
1,d4f39376166e79924e73a830012f8ea5,3.805600e+13,0.0,2020-08-05,3.131665e+11,608.0,2020-08-05 07:18:45,1.840615e+09,Apps: iPhone,60.0,...,4.122317e+05,6624528.0,2.202547e+08,NaN,NaN,0.001694,0.001872,NaN,NaN,0.006
2,ace49b041640a9c0b93ba28affff02e4,3.804762e+13,0.0,2020-07-26,4.023208e+11,31.0,2020-07-26 14:20:40,0.000000e+00,Browser: Core site,503.0,...,1.881010e+07,6174417.0,1.871884e+08,NaN,NaN,0.048301,0.100488,NaN,NaN,0.006
3,8891830716d4b9474906df10011cd54d,3.805159e+13,3.0,2020-07-31,1.743708e+11,2454.0,2020-07-31 06:52:23,1.123359e+09,Apps: iPhone,46.0,...,1.332561e+06,6367867.0,2.133461e+08,NaN,NaN,0.021025,0.006246,NaN,NaN,0.006
4,963d16581730aad74a104d04ffa8f650,3.804917e+13,3.0,2020-07-28,3.016997e+11,1.0,2020-07-28 09:24:12,0.000000e+00,Browser: Core site,18.0,...,8.696573e+04,7063671.0,2.377354e+08,NaN,NaN,0.000338,0.000366,NaN,NaN,0.006


In [4]:
# quickly filling out NaNs with zeroes, otherwise astype() won't work
df = df.fillna(0)

In [5]:
# df.columns.to_list()

# dropping unused columns:
to_drop =['ITEM_ID', 'GUID','SESSION_SKEY','SITE_ID', 'MIN_SEQNUM', 'MIN_SEQNUM', 'USER_ID', 'SEQNUM_RANK', 'show_cnt']
df = df.drop(columns = to_drop, axis = 1)

In [16]:
df['PHOTO_CNT'].head()

0    0.0
1    2.0
2    3.0
3    3.0
4    1.0
Name: PHOTO_CNT, dtype: float64

In [7]:
# Changing columns' types to make the dataframe more performant
df.dtypes.to_dict()

type_mapping = {'PLATFORM': 'category', 'stp_yn': 'int8', 'map_yn': 'int8', 'mdm_yn': 'int8', 'sme_type_dt': 'int8', 'vi_coupon_show': 'int8',\
               'vp_yn': 'int8', 'active_item_yn': 'int8', 'return_accept_yn': 'int8', 'return_duration_in_vi': 'int8',\
               'qty_available': 'int16', 'qty_sold': 'int16', 'show_view_1hr_cnt': 'int16', 'show_view_24hr_cnt': 'int16',\
               'show_watch_1hr_cnt': 'int16', 'show_watch_24hr_cnt': 'int16','show_sld_1hr_cnt': 'int16', 'show_sld_24hr_cnt': 'int16',\
               'show_last_item_cnt': 'int16', 'show_limited_time_cnt': 'int16', 'show_limited_qty_cnt': 'int16',\
               'BIN_CNT': 'int16', 'BID_CNT': 'int16','OFFER_CNT': 'int16', 'WATCH_CNT': 'int16', 'ASQ_CNT': 'int16',\
               'BBO_CNT': 'int16', 'BBOC_CNT': 'int16', 'CONV_YN': 'int8', 'MQ_YN': 'int8', 'FP_YN': 'int8', 'BO_YN': 'int8',\
               'FREE_SHIPPING_YN': 'int8', 'ITEM_CONDITION': 'category', 'ITEM_CONDITION_DETAIL': 'category', 'META_CATEG_NAME': 'category',\
               'CATEG_LVL2_NAME': 'category','VERTICAL': 'category', 'RETURN_ACCPT_YN_NEW': 'int8', 'RETURN_TYPE': 'category', 'PHOTO_CNT': 'int8',\
               }
df = df.astype(type_mapping)


In [8]:
df.dtypes.to_dict()

{'SESSION_START_DT': dtype('O'),
 'MIN_EVENT_TIMESTAMP': dtype('O'),
 'PLATFORM': CategoricalDtype(categories=['__UNKNOWN_CATEGORIES__'], ordered=False),
 'asp_usd': dtype('float64'),
 'stp_yn': dtype('int8'),
 'map_yn': dtype('int8'),
 'mdm_yn': dtype('int8'),
 'sme_type_dt': dtype('int8'),
 'vi_coupon_show': dtype('int8'),
 'vp_yn': dtype('int8'),
 'VP_PILL_CNT': dtype('float64'),
 'active_item_yn': dtype('int8'),
 'return_accept_yn': dtype('int8'),
 'return_duration_in_vi': dtype('int8'),
 'qty_available': dtype('int16'),
 'qty_sold': dtype('int16'),
 'show_cnt': dtype('O'),
 'show_view_1hr_cnt': dtype('int16'),
 'show_view_24hr_cnt': dtype('int16'),
 'show_watch_1hr_cnt': dtype('int16'),
 'show_watch_24hr_cnt': dtype('int16'),
 'show_sld_1hr_cnt': dtype('int16'),
 'show_sld_24hr_cnt': dtype('int16'),
 'show_last_item_cnt': dtype('int16'),
 'show_limited_time_cnt': dtype('int16'),
 'show_limited_qty_cnt': dtype('int16'),
 'BIN_CNT': dtype('int16'),
 'BID_CNT': dtype('int16'),
 'OFFE

In [9]:
df.head()

,SESSION_START_DT,MIN_EVENT_TIMESTAMP,PLATFORM,asp_usd,stp_yn,map_yn,mdm_yn,sme_type_dt,vi_coupon_show,vp_yn,...,CATEG_GMB_P1D_BEFORE_VI,TTL_BI_P1D_BEFORE_VI,TTL_GMB_P1D_BEFORE_VI,ITEM_BI_SHARE_IN_CATEG_P1D_BEFORE_VI,ITEM_GMB_SHARE_IN_CATEG_P1D_BEFORE_VI,CATEG_BI_SHARE_IN_TTL_P1D_BEFORE_VI,CATEG_GMB_SHARE_IN_TTL_P1D_BEFORE_VI,BI_INVERSE_FREQUENCY_SCORE,GMB_INVERSE_FREQUENCY_SCORE,rnk
0,2020-07-30,2020-07-30 17:15:51,Browser: mWeb,5.0,0,0,0,0,0,1,...,1.538424e+05,6525144.0,2.188364e+08,0.00017,0.000218,0.001804,0.000703,0.001074,0.001582,0.006
1,2020-08-05,2020-08-05 07:18:45,Apps: iPhone,60.0,0,0,0,0,0,0,...,4.122317e+05,6624528.0,2.202547e+08,0.00000,0.000000,0.001694,0.001872,0.000000,0.000000,0.006
2,2020-07-26,2020-07-26 14:20:40,Browser: Core site,503.0,0,0,0,0,0,0,...,1.881010e+07,6174417.0,1.871884e+08,0.00000,0.000000,0.048301,0.100488,0.000000,0.000000,0.006
3,2020-07-31,2020-07-31 06:52:23,Apps: iPhone,46.0,0,0,0,0,0,0,...,1.332561e+06,6367867.0,2.133461e+08,0.00000,0.000000,0.021025,0.006246,0.000000,0.000000,0.006
4,2020-07-28,2020-07-28 09:24:12,Browser: Core site,18.0,0,0,0,0,0,0,...,8.696573e+04,7063671.0,2.377354e+08,0.00000,0.000000,0.000338,0.000366,0.000000,0.000000,0.006
